In [82]:
import os
import csv
import re
import os
import pandas as pd
from difflib import get_close_matches

In [100]:
# Paths
output_dir = r"C:\Users\tishya\SELF MADE\SCRAP_UNCLE\outputs_images_each_categories"
faq_excel_path = r"C:\Users\tishya\OneDrive\Desktop\refined faq's.csv"


faq_df = pd.read_csv(faq_excel_path, encoding='latin1')
final_df = pd.DataFrame()

In [101]:


def format_question_name(raw_name):
    import re
    cleaned = re.sub(r'_+', ' ', raw_name)
    cleaned = re.sub(r'\bQ\d+\b|\(\d+\)|-\d+$', '', cleaned, flags=re.IGNORECASE)
    cleaned = re.sub(r'[?.!]+$', '', cleaned)
    return cleaned.strip().upper()

def build_final_df(output_dir, faq_df):
    final_data = []
    image_id_counter = 1

    # Make a copy and filter out rows with NaN in 'question'
    faq_df_valid = faq_df[faq_df['question'].notna()].copy()
    faq_df_valid['question_upper'] = faq_df_valid['question'].str.upper()

    for category in os.listdir(output_dir):
        category_path = os.path.join(output_dir, category)
        if os.path.isdir(category_path):
            for question in os.listdir(category_path):
                question_path = os.path.join(category_path, question)
                if os.path.isdir(question_path):
                    formatted_question = format_question_name(question)

                    # Fuzzy match with cleaned question list
                    match = get_close_matches(formatted_question, faq_df_valid['question_upper'].tolist(), n=1, cutoff=0.85)
                    if match:
                        matched_row = faq_df_valid[faq_df_valid['question_upper'] == match[0]]
                        if not matched_row.empty:
                            faq_id = matched_row.iloc[0]['id']
                            for img_file in os.listdir(question_path):
                                if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
                                    img_name = os.path.splitext(img_file)[0]
                                    final_data.append({
                                        'image_id': image_id_counter,
                                        'faq_id': faq_id,
                                        'image_url': 'link',
                                        'description': img_name
                                    })
                                    image_id_counter += 1

    final_df = pd.DataFrame(final_data, columns=['image_id', 'faq_id', 'image_url', 'description'])
    return final_df

In [102]:
final_df = build_final_df(output_dir, faq_df)
print(final_df.head())


   image_id faq_id image_url                  description
0         1      1      link           01_Access_Granted!
1         2      1      link         02_Apartment_Pickup!
2         3      2      link  01_Market_Fluctuation_Check
3         4      2      link       02_Price_Variation_Map
4         5      4      link                       step 1


In [103]:
final_df

,image_id,faq_id,image_url,description
0,1,1,link,01_Access_Granted!
1,2,1,link,02_Apartment_Pickup!
2,3,2,link,01_Market_Fluctuation_Check
3,4,2,link,02_Price_Variation_Map
4,5,4,link,step 1
...,...,...,...,...
280,281,124,link,03_Request_Confirmed
281,282,122,link,01_Scheduling_the_Pickup
282,283,122,link,02_Scrap_Collection
283,284,120,link,01_App_Access


In [104]:
final_df.to_csv("images_metadata", index=False)

In [105]:
final_df.to_excel('final_images_metadata.xlsx', index=False)
